In [2]:
import json
from abc import ABC, abstractmethod
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import random

# ------------------------ Device Abstract Class ------------------------
class Device(ABC):
    def __init__(self, name):
        self.name = name
        self.is_on = False
        self.total_hours = 0
        self.last_on_time = None

    @abstractmethod
    def turn_on(self):
        pass

    @abstractmethod
    def turn_off(self):
        pass

    @abstractmethod
    def status_report(self):
        pass

    def update_energy_usage(self):
        if self.is_on and self.last_on_time:
            delta = datetime.now() - self.last_on_time
            self.total_hours += delta.total_seconds() / 3600
            self.last_on_time = datetime.now()

    def energy_consumption(self):
        return round(self.total_hours * 0.5, 2)  # kWh

# ------------------------ Concrete Device Classes ------------------------
class Light(Device):
    def turn_on(self):
        self.is_on = True
        self.last_on_time = datetime.now()

    def turn_off(self):
        self.update_energy_usage()
        self.is_on = False

    def status_report(self):
        return f"Light: {'ON' if self.is_on else 'OFF'}, Energy: {self.energy_consumption()} kWh"

class Thermostat(Device):
    def __init__(self, name):
        super().__init__(name)
        self.temperature = 25
        self.observers = []

    def add_observer(self, device):
        self.observers.append(device)

    def set_temperature(self, temp):
        self.temperature = temp
        if temp > 30:
            for obs in self.observers:
                obs.turn_on()

    def turn_on(self):
        self.is_on = True
        self.last_on_time = datetime.now()

    def turn_off(self):
        self.update_energy_usage()
        self.is_on = False

    def status_report(self):
        return f"Thermostat: {self.temperature}°C, {'ON' if self.is_on else 'OFF'}"

class Fan(Device):
    def turn_on(self):
        if not self.is_on:
            print("Fan avtomatik yoqildi!")
        self.is_on = True
        self.last_on_time = datetime.now()

    def turn_off(self):
        self.update_energy_usage()
        self.is_on = False

    def status_report(self):
        return f"Fan: {'ON' if self.is_on else 'OFF'}, Energy: {self.energy_consumption()} kWh"

class Camera(Device):
    def turn_on(self):
        self.is_on = True
        self.last_on_time = datetime.now()

    def turn_off(self):
        self.update_energy_usage()
        self.is_on = False

    def status_report(self):
        return f"Camera: {'ON' if self.is_on else 'OFF'}"

class Alarm(Device):
    def turn_on(self):
        self.is_on = True
        self.last_on_time = datetime.now()
        print("Signalizatsiya faollashdi!")

    def turn_off(self):
        self.update_energy_usage()
        self.is_on = False
        print("Signalizatsiya o‘chirildi.")

    def status_report(self):
        return f"Alarm: {'ON' if self.is_on else 'OFF'}"

# ------------------------ Controller Class ------------------------
class SmartHomeController:
    def __init__(self):
        self.devices = {}

    def add_device(self, device):
        self.devices[device.name] = device

    def send_signal(self, name, action):
        device = self.devices.get(name)
        if device:
            if action == 'on':
                device.turn_on()
            elif action == 'off':
                device.turn_off()

    def save_state(self):
        data = {
            name: {
                'is_on': dev.is_on,
                'total_hours': dev.total_hours
            } for name, dev in self.devices.items()
        }
        with open("devices.json", "w") as f:
            json.dump(data, f, indent=4)

    def load_state(self):
        try:
            with open("devices.json", "r") as f:
                data = json.load(f)
            for name, state in data.items():
                if name in self.devices:
                    self.devices[name].is_on = state['is_on']
                    self.devices[name].total_hours = state['total_hours']
        except FileNotFoundError:
            pass

    def report(self):
        for dev in self.devices.values():
            print(dev.status_report())

# ------------------------ Temperature Graph ------------------------
def draw_temperature_chart():
    temps = [random.randint(24, 35) for _ in range(24)]
    hours = list(range(24))
    plt.plot(hours, temps, marker='o')
    plt.title("Harorat grafigi (soatlik)")
    plt.xlabel("Soat")
    plt.ylabel("Harorat (°C)")
    plt.grid(True)
    plt.show()

# ------------------------ CLI ------------------------
def main():
    controller = SmartHomeController()
    light = Light("light1")
    thermostat = Thermostat("thermo1")
    fan = Fan("fan1")
    cam = Camera("cam1")
    alarm = Alarm("alarm1")

    thermostat.add_observer(fan)

    for d in [light, thermostat, fan, cam, alarm]:
        controller.add_device(d)

    controller.load_state()

    while True:
        print("""
1. Qurilma holati
2. Qurilmani boshqarish
3. Haroratni o'zgartirish
4. Energiyani ko‘rish
5. Harorat grafigi
6. Chiqish
""")
        choice = input("Tanlang: ")
        if choice == '1':
            controller.report()
        elif choice == '2':
            name = input("Qurilma nomi: ")
            action = input("'on' yoki 'off': ")
            controller.send_signal(name, action)
        elif choice == '3':
            temp = int(input("Yangi harorat: "))
            thermostat.set_temperature(temp)
        elif choice == '4':
            for dev in controller.devices.values():
                print(f"{dev.name}: {dev.energy_consumption()} kWh")
        elif choice == '5':
            draw_temperature_chart()
        elif choice == '6':
            controller.save_state()
            break

if __name__ == "__main__":
    main()


1. Qurilma holati
2. Qurilmani boshqarish
3. Haroratni o'zgartirish
4. Energiyani ko‘rish
5. Harorat grafigi
6. Chiqish



Light: OFF, Energy: 0.0 kWh
Thermostat: 25°C, OFF
Fan: OFF, Energy: 0.0 kWh
Camera: OFF
Alarm: OFF

1. Qurilma holati
2. Qurilmani boshqarish
3. Haroratni o'zgartirish
4. Energiyani ko‘rish
5. Harorat grafigi
6. Chiqish


1. Qurilma holati
2. Qurilmani boshqarish
3. Haroratni o'zgartirish
4. Energiyani ko‘rish
5. Harorat grafigi
6. Chiqish



In [7]:
import json
from abc import ABC, abstractmethod
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import random

# ------------------------ Device Abstract Class ------------------------
class Device(ABC):
    def __init__(self, name):
        self.name = name
        self.is_on = False
        self.total_hours = 0
        self.last_on_time = None

    @abstractmethod
    def turn_on(self):
        pass

    @abstractmethod
    def turn_off(self):
        pass

    @abstractmethod
    def status_report(self):
        pass

    def update_energy_usage(self):
        if self.is_on and self.last_on_time:
            delta = datetime.now() - self.last_on_time
            self.total_hours += delta.total_seconds() / 3600
            self.last_on_time = datetime.now()

    def energy_consumption(self):
        return round(self.total_hours * 0.5, 2)  # kWh

# ------------------------ Concrete Device Classes ------------------------
class Light(Device):
    def turn_on(self):
        self.is_on = True
        self.last_on_time = datetime.now()

    def turn_off(self):
        self.update_energy_usage()
        self.is_on = False

    def status_report(self):
        return f"Light: {'ON' if self.is_on else 'OFF'}, Energy: {self.energy_consumption()} kWh"

class Thermostat(Device):
    def __init__(self, name):
        super().__init__(name)
        self.temperature = 25
        self.observers = []

    def add_observer(self, device):
        self.observers.append(device)

    def set_temperature(self, temp):
        self.temperature = temp
        if temp > 30:
            for obs in self.observers:
                obs.turn_on()

    def turn_on(self):
        self.is_on = True
        self.last_on_time = datetime.now()

    def turn_off(self):
        self.update_energy_usage()
        self.is_on = False

    def status_report(self):
        return f"Thermostat: {self.temperature}°C, {'ON' if self.is_on else 'OFF'}"

class Fan(Device):
    def turn_on(self):
        if not self.is_on:
            print("Fan avtomatik yoqildi!")
        self.is_on = True
        self.last_on_time = datetime.now()

    def turn_off(self):
        self.update_energy_usage()
        self.is_on = False

    def status_report(self):
        return f"Fan: {'ON' if self.is_on else 'OFF'}, Energy: {self.energy_consumption()} kWh"

class Camera(Device):
    def turn_on(self):
        self.is_on = True
        self.last_on_time = datetime.now()

    def turn_off(self):
        self.update_energy_usage()
        self.is_on = False

    def status_report(self):
        return f"Camera: {'ON' if self.is_on else 'OFF'}"

class Alarm(Device):
    def turn_on(self):
        self.is_on = True
        self.last_on_time = datetime.now()
        print("Signalizatsiya faollashdi!")

    def turn_off(self):
        self.update_energy_usage()
        self.is_on = False
        print("Signalizatsiya o‘chirildi.")

    def status_report(self):
        return f"Alarm: {'ON' if self.is_on else 'OFF'}"

# ------------------------ Controller Class ------------------------
class SmartHomeController:
    def __init__(self):
        self.devices = {}

    def add_device(self, device):
        self.devices[device.name.lower()] = device

    def send_signal(self, name, action):
        device = self.devices.get(name.lower())
        if device:
            if action.lower() == 'on':
                device.turn_on()
            elif action.lower() == 'off':
                device.turn_off()
            else:
                print("Noto‘g‘ri amal kiritildi. Faqat 'on' yoki 'off'.")
        else:
            print("Bunday nomli qurilma topilmadi.")

    def save_state(self):
        data = {
            name: {
                'is_on': dev.is_on,
                'total_hours': dev.total_hours
            } for name, dev in self.devices.items()
        }
        with open("devices.json", "w") as f:
            json.dump(data, f, indent=4)

    def load_state(self):
        try:
            with open("devices.json", "r") as f:
                data = json.load(f)
            for name, state in data.items():
                if name in self.devices:
                    self.devices[name].is_on = state['is_on']
                    self.devices[name].total_hours = state['total_hours']
        except FileNotFoundError:
            pass

    def report(self):
        for dev in self.devices.values():
            print(dev.status_report())

# ------------------------ Temperature Graph ------------------------
def draw_temperature_chart():
    temps = [random.randint(24, 35) for _ in range(24)]
    hours = list(range(24))
    plt.plot(hours, temps, marker='o')
    plt.title("Harorat grafigi (soatlik)")
    plt.xlabel("Soat")
    plt.ylabel("Harorat (°C)")
    plt.grid(True)
    plt.show()

# ------------------------ CLI ------------------------
def main():
    controller = SmartHomeController()
    light = Light("light1")
    thermostat = Thermostat("thermo1")
    fan = Fan("fan1")
    cam = Camera("cam1")
    alarm = Alarm("alarm1")

    thermostat.add_observer(fan)

    for d in [light, thermostat, fan, cam, alarm]:
        controller.add_device(d)

    controller.load_state()

    while True:
        print("""
1. Qurilma holati
2. Qurilmani boshqarish
3. Haroratni o'zgartirish
4. Energiyani ko‘rish
5. Harorat grafigi
6. Chiqish
""")
        choice = input("Tanlang: ")
        if choice == '1':
            controller.report()
        elif choice == '2':
            print("Mavjud qurilmalar:", ", ".join(controller.devices.keys()))
            name = input("Qurilma nomi: ")
            action = input("'on' yoki 'off': ")
            controller.send_signal(name, action)
        elif choice == '3':
            try:
                temp = int(input("Yangi harorat: "))
                thermostat.set_temperature(temp)
            except ValueError:
                print("Iltimos, to‘g‘ri raqam kiriting.")
        elif choice == '4':
            for name, dev in controller.devices.items():
                print(f"{name}: {dev.energy_consumption()} kWh")
        elif choice == '5':
            draw_temperature_chart()
        elif choice == '6':
            controller.save_state()
            break

if __name__ == "__main__":
    main()



1. Qurilma holati
2. Qurilmani boshqarish
3. Haroratni o'zgartirish
4. Energiyani ko‘rish
5. Harorat grafigi
6. Chiqish



Mavjud qurilmalar: light1, thermo1, fan1, cam1, alarm1
Noto‘g‘ri amal kiritildi. Faqat 'on' yoki 'off'.

1. Qurilma holati
2. Qurilmani boshqarish
3. Haroratni o'zgartirish
4. Energiyani ko‘rish
5. Harorat grafigi
6. Chiqish


1. Qurilma holati
2. Qurilmani boshqarish
3. Haroratni o'zgartirish
4. Energiyani ko‘rish
5. Harorat grafigi
6. Chiqish


1. Qurilma holati
2. Qurilmani boshqarish
3. Haroratni o'zgartirish
4. Energiyani ko‘rish
5. Harorat grafigi
6. Chiqish

light1: 0.0 kWh
thermo1: 0.0 kWh
fan1: 0.0 kWh
cam1: 0.0 kWh
alarm1: 0.0 kWh

1. Qurilma holati
2. Qurilmani boshqarish
3. Haroratni o'zgartirish
4. Energiyani ko‘rish
5. Harorat grafigi
6. Chiqish

